# 01 신경망의 개요

코드 1-1 [1] 필요한 패키지 불러오기

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris

코드 1-1 [2] 데이터 준비 함수 정의하기

In [23]:
def prepare_data(target):
    iris = load_iris()          # iris data set 읽기
    X_tr = iris.data[:, 2:]     # 4개의 특징 중 꽃잎의 길이와 폭 선택
    labels = iris.target_names  # 'setosa', 'versicolor', 'vriginica'
    y = iris.target
    
    # 학습표본의 레이블 지정 - target에 지정된 레이블이면 1, 그 외는 0
    target_lable = np.argwhere(labels == target)
    y_tr = (y == target_lable).astype(int)
    return X_tr, y_tr, ['(1) '+target, '(0) the others']

코드 1-1 [3] 활성화 함수 - 단위 계단 함수

In [1]:
def step(x):
    return int(x >= 0)

코드 1-1 [4] 퍼셉트론 클래스 선언

In [3]:
class Perceptron:
    def __init__(self, dim, activation):
        rnd = np.random.default_rng()
        self.dim = dim
        self.activation = activation
        # 가중치(w)와 바이어스(b)를 He normal 방식으로 초기화
        self.w = rnd.normal(scale=np.sart(2.0 / dim), size=dim)
        self.b = rnd.normal(scale=np.sqrt(2.0 / dim))

def printW(self):
    for i in range(self.dim):
        print('  w{} = {:6.3f}'.format(i+1, self.w[i]), end='')
    print('  b = {:6.3f}'.format(self.b))

def predict(self, x) : # numpy 배열 x에 저장된 표본의 출력 계산
    return np.array([self.activation(np.dot(self.w, x[i]) + self.b) 
                                     for i in range(len(x))])

def fit(self, X, y, N, epochs, eta=0.01):
    # 학습표본의 인덱스를 무작위 순서로 섞음
    idx = list(range (N))
    np.random.shuffle(idx)
    X = np.array([X[idx[i]] for i in range (N)])
    y = np.array([y[idx[i]] for i in range (N)])
    f = 'Epochs = {:4d}    Loss = {:8.5f}'
    print('w의 초깃값 ', end='')
    self.printw()
    for j in range(epochs):
        for i in range(N):
            # x[1]에 대한 출력 오차 계산
            delta = self.predict([X[i]])[0] - y[i]
            self.w -= eta * delta * X[i]
            self.b -= eta * delta
        # 학습 과정 출력
        if j < 10 or (j+1) % 100 == 0:
            loss = self.predict(X) - y
            loss = (loss * loss).sum() / N
            print(f.format(j+1, loss), end='')
            self.printW()